In [83]:
# 1. 표준 라이브러리
import sys
from pathlib import Path
import platform
import time
import json
import os
import joblib
from datetime import datetime
from typing import Optional
from dataclasses import dataclass


# 2. 서드파티 라이브러리 

# 2-1. 시각화
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns

# 2-2. 
import shap
from imblearn.pipeline import Pipeline as ImbPipeline
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, make_scorer, recall_score, 
    precision_score, f1_score, fbeta_score, average_precision_score, balanced_accuracy_score, precision_recall_curve
)
from scipy.stats import uniform, randint
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.base import BaseEstimator
from sklearn.impute import KNNImputer

sys.path.append(str(Path.cwd().parent))
from utils import DATA_DIR, MODEL_DIR

In [84]:
# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

mpl.rcParams['axes.unicode_minus'] = False
# 폰트 개인 경로에 맞춰서 변경
# FONT_DIR = Path("/path/to/fonts")
# font_path = FONT_DIR / 'FREESENTATION-6SEMIBOLD.ttf'
# prop = fm.FontProperties(fname=font_path)

In [85]:
pandas_kwargs = {
    'parse_dates': ['검정일자'],
    'date_format': '%Y-%m-%d'
}

milk: pd.DataFrame = pd.read_csv(DATA_DIR /'interim' / 'milk.csv', **pandas_kwargs)
milk.head()

,농장아이디,개체번호,검정일자,누적착유일(연계),무지고형분,MUN,305일유량,305일무지고형분,전산차비유지속성,전산차건유전유량,...,농장구분,정액코드분류,가격,가격미달,가격구분,분만간격,분만월령,공태일수_log,계절_sin,비유단계_sin
0,20249,20120709020022,2020-01-11,154,0,1,9714.0,805.0,50.5,16.1,...,1,NaN,1056.34,0,1,NaN,85,NaN,-2.449294e-16,1.000000e+00
1,20249,20120504020095,2020-01-11,52,0,1,NaN,NaN,68.8,22.6,...,1,NaN,1056.34,0,1,NaN,90,NaN,-2.449294e-16,1.000000e+00
2,20249,20111008020210,2020-01-11,115,0,1,11008.0,971.0,54.1,24.5,...,1,NaN,1060.34,0,1,NaN,95,NaN,-2.449294e-16,1.224647e-16
3,20249,20121014020049,2020-01-11,290,1,1,11318.0,1011.0,58.6,37.8,...,1,7HO,1062.31,0,1,NaN,77,5.4161,-2.449294e-16,-1.000000e+00
4,20249,20130812020216,2020-01-11,100,0,1,11324.0,957.0,68.1,20.5,...,1,NaN,1056.34,0,1,NaN,75,NaN,-2.449294e-16,1.224647e-16


In [86]:
df = milk

In [87]:
features = [
    '전산차건유전유량', 
    '산차', 
    '농후사료급여량(연계)', 
    '공태일수', 
    '계절', 
    '농장구분', 
    '분만간격',
    # '정액코드분류'
]

target = '가격미달'

## 범주형 처리

In [88]:
# categorical_features = ['정액코드분류']

# for categorical_feature in categorical_features:
#     df[categorical_feature] = df[categorical_feature].astype('category')

## Train Test Split

In [89]:
split_date = '2021-08-01'
train = df[df['검정일자'] < split_date]
test = df[df['검정일자'] >= split_date]

X_train, X_test = train[features], test[features]
y_train, y_test = train[target], test[target]

In [90]:
train.shape, test.shape

((31743, 26), (8521, 26))

In [91]:

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pkg_resources")

# ============================================
# 1. 클래스 비율 확인
# ============================================
print("="*60)
print("클래스 분포 분석")
print("="*60)
print(y_train.value_counts().sort_index())
print(f"\n비율:")
print(y_train.value_counts(normalize=True).sort_index())

# ✅ 수정: 0이 일반가(다수), 1이 가격미달(소수)
majority_count = (y_train == 0).sum()   # 일반가 (다수) - 약 90%
minority_count = (y_train == 1).sum()   # 가격미달 (소수) - 약 10%

print(f"\n클래스 구성:")
print(f"   0 (일반가, 다수): {majority_count}개 ({majority_count/len(y_train)*100:.1f}%)")
print(f"   1 (가격미달, 소수): {minority_count}개 ({minority_count/len(y_train)*100:.1f}%)")

# ✅ scale_pos_weight: 양성 클래스(1=가격미달=소수)에 대한 가중치
# 공식: (다수 클래스 개수) / (소수 클래스 개수)
scale_pos_weight = majority_count / minority_count

print(f"\n계산된 scale_pos_weight: {scale_pos_weight:.2f}")
print(f"   → LightGBM은 자동으로 클래스 1(가격미달)에 이 가중치 적용")

# 결측치 확인
print("\n결측치 현황:")
missing_info = pd.DataFrame({
    'Feature': X_train.columns,
    '결측_개수': X_train.isnull().sum().values,
    '결측_비율(%)': (X_train.isnull().mean() * 100).values
}).sort_values('결측_비율(%)', ascending=False)
print(missing_info.head(10))
print(f"\n전체 결측 비율: {X_train.isnull().mean().mean()*100:.1f}%")
print(f"Feature 개수: {X_train.shape[1]}개")


클래스 분포 분석
가격미달
0    28406
1     3337
Name: count, dtype: int64

비율:
가격미달
0    0.894874
1    0.105126
Name: proportion, dtype: float64

클래스 구성:
   0 (일반가, 다수): 28406개 (89.5%)
   1 (가격미달, 소수): 3337개 (10.5%)

계산된 scale_pos_weight: 8.51
   → LightGBM은 자동으로 클래스 1(가격미달)에 이 가중치 적용

결측치 현황:
       Feature  결측_개수   결측_비율(%)
6         분만간격  20733  65.315188
3         공태일수  13326  41.980909
0     전산차건유전유량  12489  39.344107
2  농후사료급여량(연계)      0   0.000000
1           산차      0   0.000000
4           계절      0   0.000000
5         농장구분      0   0.000000

전체 결측 비율: 20.9%
Feature 개수: 7개


In [92]:
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.impute import SimpleImputer

In [93]:
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import BorderlineSMOTE
from lightgbm import LGBMClassifier
from scipy.stats import randint, uniform

# ============================================
# Pipeline 구성 (강력한 안전장치)
# ============================================
pipeline = ImbPipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('smote', BorderlineSMOTE(
        random_state=42,
        sampling_strategy=0.5,  # 🔥 'auto' 대신 0.5 (다수 클래스의 50%만 생성)
        kind='borderline-1',
        k_neighbors=5,
        m_neighbors=10
    )),
    ('model', LGBMClassifier(
        objective='binary',
        device='cpu',  # 🔥 CPU 모드 (안정적)
        random_state=42,
        verbosity=-1,
        n_jobs=-1,
        
        # 🔥🔥🔥 핵심 안전 파라미터
        min_child_samples=100,     # 기본값 20 → 100 (대폭 증가)
        min_data_in_leaf=100,      # 기본값 20 → 100
        min_split_gain=0.01,       # 최소 분할 이득
        min_sum_hessian_in_leaf=10,  # Hessian 합 최소값
        
        # 트리 제한
        max_depth=5,               # 깊이 제한
        num_leaves=31,             # 리프 제한
        
        # 정규화
        reg_alpha=0.1,
        reg_lambda=0.1,
        
        # 기타
        boost_from_average=False,
        force_col_wise=True,
        metric='None',
    ))
])

print("✅ Pipeline 구성 완료 (강력한 안전장치):")
print("   - min_child_samples=100")
print("   - min_data_in_leaf=100")
print("   - sampling_strategy=0.5 (과도한 SMOTE 방지)")

✅ Pipeline 구성 완료 (강력한 안전장치):
   - min_child_samples=100
   - min_data_in_leaf=100
   - sampling_strategy=0.5 (과도한 SMOTE 방지)


In [99]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

# 🔥 DataFrame을 유지하는 Imputer 래퍼
class DataFrameImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='median'):
        self.strategy = strategy
        self.imputer = None
        self.feature_names = None
    
    def fit(self, X, y=None):
        self.feature_names = X.columns.tolist() if isinstance(X, pd.DataFrame) else None
        self.imputer = SimpleImputer(strategy=self.strategy)
        self.imputer.fit(X)
        return self
    
    def transform(self, X):
        X_imputed = self.imputer.transform(X)
        
        # DataFrame으로 변환하여 반환
        if self.feature_names is not None:
            return pd.DataFrame(X_imputed, columns=self.feature_names, index=X.index)
        return X_imputed

# ============================================
# 2. Pipeline 구성 (BorderlineSMOTE + LightGBM)
# ============================================
print("\n" + "="*60)
print("Pipeline 구성")
print("="*60)

pipeline = ImbPipeline([
    ('imputer', DataFrameImputer(strategy='median')),
    # 2. BorderlineSMOTE
    ('smote', BorderlineSMOTE(
        random_state=42,
        sampling_strategy='auto',
        kind='borderline-1',  # borderline-1이 더 안정적
        k_neighbors=5,
        m_neighbors=10
    )),
    ('model', LGBMClassifier(
        objective='binary',
        device='cpu',  # 'cpu'로 변경 가능
        random_state=42,
        verbosity=-1,
        n_jobs=5,
        boost_from_average=False,
        force_col_wise=True,
        metric='None',
    ))
])


print("✅ Pipeline 구성 완료:")
print("   1. BorderlineSMOTE")
print(f"      - kind: borderline-1 (경계선 샘플 중심 오버샘플링)")
print(f"      - sampling_strategy: auto (소수 클래스를 다수 클래스 수만큼 증강)")
print("   2. LGBMClassifier (GPU)")
print(f"      - scale_pos_weight는 파라미터 탐색에 포함")

# ============================================
# 3. 파라미터 분포
# ============================================
print(f"\n소수 클래스(1=가격미달) 샘플 수: {minority_count}개")
print("BorderlineSMOTE 적용 시 증강될 예정")

param_distributions = {
    # 🔥 BorderlineSMOTE 파라미터
    'smote__k_neighbors': randint(3, 10),        # 경계선 탐지용
    'smote__m_neighbors': randint(5, 15),        # 합성 샘플 생성용
    'smote__kind': ['borderline-1', 'borderline-2'],  # 경계선 타입
    
    # 🔥 LightGBM 파라미터
    'model__n_estimators': randint(600, 1000),
    'model__learning_rate': uniform(0.005, 0.045),
    'model__max_depth': randint(3, 8),
    'model__num_leaves': randint(15, 80),
    'model__min_child_samples': randint(5, 50),
    'model__subsample': uniform(0.6, 0.4),
    'model__colsample_bytree': uniform(0.6, 0.4),
    'model__min_child_weight': [1e-5, 1e-4, 1e-3, 1e-2],
    'model__reg_alpha': uniform(0, 0.5),
    'model__reg_lambda': uniform(0, 0.5),
    'model__scale_pos_weight': uniform(1.0, 3.0),  # SMOTE 쓰면 낮게 설정
}

# ============================================
# 4. Scoring 설정
# ============================================
scoring = {
    'recall': make_scorer(recall_score),
    'f2': make_scorer(fbeta_score, beta=2),
    'pr_auc': make_scorer(average_precision_score, response_method='predict_proba'),
    'precision': make_scorer(precision_score, zero_division=0),
    'f1': make_scorer(f1_score, zero_division=0),
    'roc_auc': 'roc_auc'
}

main_score = 'f1'  # 🔥 F1으로 변경 (precision/recall 균형)

print("\n" + "="*60)
print("Scoring 전략")
print("="*60)
print(f"✅ 메인 목표: {main_score.upper()}")
print(f"   타깃: 클래스 1 (가격미달) 검출 최적화")
print(f"   SMOTE로 균형 맞춘 후 F1 최대화")

# ============================================
# 5. Stratified K-Fold
# ============================================
n_folds = 5
n_iter = 300  # 🔥 SMOTE는 시간이 오래 걸리므로 100으로 축소
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

print(f"\n교차검증 Folds: {n_folds}")
print(f"랜덤 샘플링 조합 수: {n_iter}개")
print(f"총 Fits: {n_iter} × {n_folds} = {n_iter * n_folds}")
print(f"⚠️  BorderlineSMOTE는 연산 시간이 오래 걸립니다!")

# ============================================
# 6. RandomizedSearchCV 실행
# ============================================
print("\n" + "="*60)
print("🚀 RandomizedSearchCV 시작!")
print("="*60)

lgbm_random = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=cv,
    scoring=scoring,
    refit=main_score,
    verbose=3,
    random_state=42,
    n_jobs=1,  # 🔥 Pipeline은 n_jobs=1 권장 (내부에서 병렬화)
    error_score=np.nan
)

start_actual = time.time()
lgbm_random.fit(X_train, y_train)
actual_time = time.time() - start_actual

# ============================================
# 7. 결과 출력
# ============================================
print("\n" + "="*60)
print("✅ 완료!")
print("="*60)
print(f"⏱️  소요 시간: {actual_time/60:.1f}분 ({actual_time:.0f}초)")

print(f"\n🏆 최고 {main_score.upper()} (CV): {lgbm_random.best_score_:.4f}")

print("\n📋 최적 파라미터:")
for key, value in sorted(lgbm_random.best_params_.items()):
    if isinstance(value, float):
        print(f"   {key}: {value:.4f}")
    else:
        print(f"   {key}: {value}")




Pipeline 구성
✅ Pipeline 구성 완료:
   1. BorderlineSMOTE
      - kind: borderline-1 (경계선 샘플 중심 오버샘플링)
      - sampling_strategy: auto (소수 클래스를 다수 클래스 수만큼 증강)
   2. LGBMClassifier (GPU)
      - scale_pos_weight는 파라미터 탐색에 포함

소수 클래스(1=가격미달) 샘플 수: 3337개
BorderlineSMOTE 적용 시 증강될 예정

Scoring 전략
✅ 메인 목표: F1
   타깃: 클래스 1 (가격미달) 검출 최적화
   SMOTE로 균형 맞춘 후 F1 최대화

교차검증 Folds: 5
랜덤 샘플링 조합 수: 300개
총 Fits: 300 × 5 = 1500
⚠️  BorderlineSMOTE는 연산 시간이 오래 걸립니다!

🚀 RandomizedSearchCV 시작!
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
[CV 1/5] END model__colsample_bytree=0.749816047538945, model__learning_rate=0.047782143788446224, model__max_depth=5, model__min_child_samples=12, model__min_child_weight=1e-05, model__n_estimators=620, model__num_leaves=38, model__reg_alpha=0.32544423647442644, model__reg_lambda=0.028205789513550128, model__scale_pos_weight=3.165996316800474, model__subsample=0.9754210836063001, smote__k_neighbors=4, smote__kind=borderline-2, smote__m_neighbors=9; f1: (test=0.

In [95]:
# ============================================
# 8. 전체 결과 비교
# ============================================
results_df = pd.DataFrame(lgbm_random.cv_results_)

scoring_cols = [f'mean_test_{score}' for score in scoring.keys()]
comparison_cols = [f'rank_test_{main_score}'] + scoring_cols + ['params']

print("\n" + "="*60)
print(f"📊 상위 10개 파라미터 조합 ({main_score.upper()} 기준)")
print("="*60)
top_10 = results_df[comparison_cols].sort_values(f'rank_test_{main_score}').head(10)

for idx, row in top_10.iterrows():
    rank = int(row[f'rank_test_{main_score}'])
    pr_auc = row[f'mean_test_pr_auc']
    recall = row['mean_test_recall']
    f2 = row['mean_test_f2']
    precision = row['mean_test_precision']
    f1 = row['mean_test_f1']
    
    print(f"\n#{rank} - {main_score.upper()}: {row[f'mean_test_{main_score}']:.4f}")
    print(f"   Recall: {recall:.4f} | Precision: {precision:.4f} | F2: {f2:.3f}")
    
    params = row['params']
    print(f"   SMOTE: k_neighbors={params.get('smote__k_neighbors', 'N/A')}, "
          f"kind={params.get('smote__kind', 'N/A')}")
    print(f"   Model: lr={params.get('model__learning_rate', 0):.4f}, "
          f"depth={params.get('model__max_depth', 0)}, "
          f"leaves={params.get('model__num_leaves', 0)}")

# ============================================
# 9. 테스트 세트 평가
# ============================================
print("\n" + "="*60)
print("🧪 테스트 세트 평가")
print("="*60)

best_model = lgbm_random.best_estimator_
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

# 클래스 1(가격미달)의 확률 추출
y_pred_proba_minority = y_pred_proba[:, 1]

print(classification_report(
    y_test, y_pred,
    target_names=['0 (일반가, 다수)', '1 (가격미달, 소수)']
))

print("\n혼동 행렬:")
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm,
                     index=['실제_0 (일반가)', '실제_1 (가격미달)'],
                     columns=['예측_0 (일반가)', '예측_1 (가격미달)'])
print(cm_df)

tn, fp, fn, tp = cm.ravel()

print("\n💼 비즈니스 해석 (소수 클래스=1=가격미달 기준):")
print(f"   ✅ 정확한 분류: {tn + tp}개 ({(tn+tp)/len(y_test)*100:.1f}%)")
print(f"   ⚠️  오탐 (FP): {fp}개 → 일반가를 가격미달로 오판")
print(f"   🚨 미탐 (FN): {fn}개 → 가격미달을 일반가로 오판 (치명적!)")

if tp + fn > 0:
    recall_pct = tp/(tp+fn)*100
    print(f"\n   📊 소수 클래스(1=가격미달) Recall: {recall_pct:.1f}%")
    print(f"      → 가격미달 {tp+fn}개 중 {tp}개 검출 ({fn}개 놓침)")

if tp + fp > 0:
    precision_pct = tp/(tp+fp)*100
    print(f"   📊 소수 클래스(1=가격미달) Precision: {precision_pct:.1f}%")
    print(f"      → 가격미달 판정 {tp+fp}개 중 {tp}개 정답")

test_roc_auc = roc_auc_score(y_test, y_pred_proba_minority)
print(f"   📊 ROC-AUC: {test_roc_auc:.4f}")

# ============================================
# 10. 임계값 조정 (F1 최대화)
# ============================================
print("\n" + "="*60)
print("🎯 임계값 후처리 (F1 Score 최대화)")
print("="*60)

thr_list = np.round(np.arange(0.1, 1.0, 0.01), 2)
rows = []
best_f1_thr = 0
best_threshold = 0.5

for thr in thr_list:
    y_hat = (y_pred_proba_minority >= thr).astype(int)
    
    prec = precision_score(y_test, y_hat, zero_division=0)
    rec = recall_score(y_test, y_hat, zero_division=0)
    f1 = f1_score(y_test, y_hat, zero_division=0)
    
    rows.append([thr, prec, rec, f1])
    
    if f1 > best_f1_thr:
        best_f1_thr = f1
        best_threshold = thr

thr_table = pd.DataFrame(rows, columns=['threshold', 'precision', 'recall', 'f1'])

print(f"🏆 최적 임계값: {best_threshold}")
print(f"   → F1 Score: {best_f1_thr:.4f}")

y_pred_optimal = (y_pred_proba_minority >= best_threshold).astype(int)

print(f"\n기본 (0.5) vs 최적 ({best_threshold}):")
y_pred_default = (y_pred_proba_minority >= 0.5).astype(int)

print(f"\n기본 임계값 (0.5):")
print(f"   Precision: {precision_score(y_test, y_pred_default, zero_division=0):.4f}")
print(f"   Recall:    {recall_score(y_test, y_pred_default):.4f}")
print(f"   F1:        {f1_score(y_test, y_pred_default, zero_division=0):.4f}")

print(f"\n최적 임계값 ({best_threshold}):")
print(f"   Precision: {precision_score(y_test, y_pred_optimal, zero_division=0):.4f}")
print(f"   Recall:    {recall_score(y_test, y_pred_optimal):.4f}")
print(f"   F1:        {f1_score(y_test, y_pred_optimal, zero_division=0):.4f}")

# 임계값 조정 효과
cm_opt = confusion_matrix(y_test, y_pred_optimal)
tn_opt, fp_opt, fn_opt, tp_opt = cm_opt.ravel()

print(f"\n임계값 조정 효과:")
print(f"   미탐(FN): {fn} → {fn_opt} (변화: {fn_opt-fn:+d})")
print(f"   오탐(FP): {fp} → {fp_opt} (변화: {fp_opt-fp:+d})")

# ============================================
# 11. 변수 중요도 확인
# ============================================
print("\n" + "="*60)
print("📊 변수 중요도 Top 20")
print("="*60)

final_model = best_model.named_steps['model']
feature_names = list(X_train.columns)

n_features_used = len(final_model.feature_importances_)
feature_names = feature_names[:n_features_used]

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print(importance_df.head(20).to_string(index=False))

# ============================================
# 12. 교차검증 vs 테스트 성능 비교
# ============================================
print("\n" + "="*60)
print("📈 성능 비교: 교차검증 vs 테스트")
print("="*60)

test_metrics = {
    'PR-AUC': average_precision_score(y_test, y_pred_proba_minority),
    'Recall': recall_score(y_test, y_pred_optimal),
    'F2': fbeta_score(y_test, y_pred_optimal, beta=2),
    'Precision': precision_score(y_test, y_pred_optimal, zero_division=0),
    'F1': f1_score(y_test, y_pred_optimal, zero_division=0),
    'ROC-AUC': test_roc_auc
}

print(f"{'지표':<15} {'교차검증 (CV)':<20} {'테스트':<15} {'차이':<10}")
print("-" * 60)

for metric in ['PR-AUC', 'Recall', 'F2', 'Precision', 'F1', 'ROC-AUC']:
    cv_key = metric.lower().replace('-', '_')
    cv_score = results_df.loc[results_df[f'rank_test_{main_score}'] == 1, f'mean_test_{cv_key}'].values[0]
    test_score = test_metrics[metric]
    diff = test_score - cv_score
    
    diff_str = f"{diff:+.4f}"
    if abs(diff) > 0.05:
        diff_str += " ⚠️"
    
    print(f"{metric:<15} {cv_score:<20.4f} {test_score:<15.4f} {diff_str}")

# ============================================
# 13. 최종 요약
# ============================================
print("\n" + "="*60)
print(f"📝 최종 요약 ({main_score.upper()} 최적화 + BorderlineSMOTE)")
print("="*60)
print(f"✅ 최적 모델 {main_score.upper()} (CV): {lgbm_random.best_score_:.4f}")
print(f"✅ 테스트 F1: {test_metrics['F1']:.4f} (최적 임계값 적용)")
print(f"✅ 테스트 Recall: {test_metrics['Recall']*100:.1f}%")
print(f"✅ 테스트 Precision: {test_metrics['Precision']*100:.1f}%")
print(f"✅ 최적 임계값: {best_threshold}")
print(f"✅ 치명적 오류(미탐 FN): {fn_opt}개 / {tp_opt+fn_opt}개")
print(f"⏱️  총 소요 시간: {actual_time/60:.1f}분")

print("\n💡 BorderlineSMOTE 효과:")
print(f"   - 원본 소수 클래스: {minority_count}개")
print(f"   - SMOTE 적용으로 경계선 샘플 중심 증강")
print(f"   - 최종 Recall: {test_metrics['Recall']*100:.1f}%")

# ============================================
# 14. 모델 + 파라미터 저장
# ============================================
print("\n" + "="*60)
print("💾 모델 및 파라미터 저장")
print("="*60)

save_model_input = input("모델을 저장하시겠습니까? (y/n): ")
if save_model_input.lower() == 'y':
    
    MODEL_DIR = Path('.')
    MODEL_DIR.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    model_filename = MODEL_DIR / f'lgbm_borderline_smote_{main_score}_{timestamp}.pkl'
    params_filename = MODEL_DIR / f'best_params_{timestamp}.json'
    metrics_filename = MODEL_DIR / f'test_metrics_{timestamp}.json'
    results_filename = MODEL_DIR / f'cv_results_{timestamp}.csv'
    importance_filename = MODEL_DIR / f'feature_importance_{timestamp}.csv'
    threshold_filename = MODEL_DIR / f'threshold_table_{timestamp}.csv'
    
    # 1. 모델 저장
    joblib.dump(best_model, model_filename)
    print(f"✅ 모델 저장: {model_filename}")
    
    # 2. 파라미터 저장
    best_params_serializable = {}
    for key, value in lgbm_random.best_params_.items():
        if isinstance(value, (np.integer, np.floating)):
            best_params_serializable[key] = float(value) if isinstance(value, np.floating) else int(value)
        else:
            best_params_serializable[key] = value
    
    params_info = {
        'best_params': best_params_serializable,
        f'best_{main_score}_score': float(lgbm_random.best_score_),
        'optimization_target': main_score,
        'cv_folds': n_folds,
        'n_iter': n_iter,
        'timestamp': timestamp,
        'class_labels': {
            '0': '일반가 (다수)',
            '1': '가격미달 (소수)'
        },
        'minority_count': int(minority_count),
        'majority_count': int(majority_count),
        'scale_pos_weight': float(scale_pos_weight),
        'optimal_threshold': float(best_threshold),
        'features': features,
        'split_date': split_date,
        'smote_type': 'BorderlineSMOTE'
    }
    
    with open(params_filename, 'w', encoding='utf-8') as f:
        json.dump(params_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 파라미터 저장: {params_filename}")
    
    # 3. 테스트 성능 저장
    test_metrics_serializable = {
        key: float(value) if isinstance(value, (np.floating, np.integer)) else value
        for key, value in test_metrics.items()
    }
    
    metrics_info = {
        'test_metrics': test_metrics_serializable,
        'confusion_matrix': {
            'tn': int(tn_opt),
            'fp': int(fp_opt),
            'fn': int(fn_opt),
            'tp': int(tp_opt)
        },
        'business_interpretation': {
            'total_minority': int(tp_opt + fn_opt),
            'detected_minority': int(tp_opt),
            'missed_minority': int(fn_opt),
            'recall_percentage': float(tp_opt/(tp_opt+fn_opt)*100) if (tp_opt+fn_opt) > 0 else 0,
            'false_alarms': int(fp_opt)
        },
        'optimal_threshold': float(best_threshold),
        'split_date': split_date,
        'timestamp': timestamp
    }
    
    with open(metrics_filename, 'w', encoding='utf-8') as f:
        json.dump(metrics_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 테스트 성능 저장: {metrics_filename}")
    
    # 4. CV 결과 저장
    results_df.to_csv(results_filename, index=False, encoding='utf-8')
    print(f"✅ CV 결과 저장: {results_filename}")
    
    # 5. 변수 중요도 저장
    importance_df.to_csv(importance_filename, index=False)
    print(f"✅ 변수 중요도 저장: {importance_filename}")
    
    # 6. 임계값 테이블 저장
    thr_table.to_csv(threshold_filename, index=False)
    print(f"✅ 임계값 테이블 저장: {threshold_filename}")
    
    print("\n📦 저장된 파일:")
    print(f"1. 모델: {model_filename.name}")
    print(f"2. 파라미터: {params_filename.name}")
    print(f"3. 성능: {metrics_filename.name}")
    print(f"4. CV결과: {results_filename.name}")
    print(f"5. 변수 중요도: {importance_filename.name}")
    print(f"6. 임계값 테이블: {threshold_filename.name}")
    
else:
    print("모델 저장 생략")

print("\n" + "="*60)
print("🎉 모든 작업 완료!")
print("="*60)


📊 상위 10개 파라미터 조합 (PRECISION 기준)

#1 - PRECISION: 0.5389
   Recall: 0.1930 | Precision: 0.5389 | F2: 0.221
   SMOTE: k_neighbors=9, kind=borderline-1
   Model: lr=0.0491, depth=7, leaves=57

#2 - PRECISION: 0.4831
   Recall: 0.1897 | Precision: 0.4831 | F2: 0.216
   SMOTE: k_neighbors=6, kind=borderline-1
   Model: lr=0.0290, depth=7, leaves=58

#3 - PRECISION: 0.4757
   Recall: 0.1121 | Precision: 0.4757 | F2: 0.132
   SMOTE: k_neighbors=9, kind=borderline-1
   Model: lr=0.0297, depth=5, leaves=47

#4 - PRECISION: 0.4696
   Recall: 0.0147 | Precision: 0.4696 | F2: 0.018
   SMOTE: k_neighbors=5, kind=borderline-1
   Model: lr=0.0232, depth=3, leaves=50

#5 - PRECISION: 0.4626
   Recall: 0.1807 | Precision: 0.4626 | F2: 0.206
   SMOTE: k_neighbors=7, kind=borderline-2
   Model: lr=0.0449, depth=5, leaves=23

#6 - PRECISION: 0.4528
   Recall: 0.2751 | Precision: 0.4528 | F2: 0.298
   SMOTE: k_neighbors=4, kind=borderline-1
   Model: lr=0.0479, depth=7, leaves=77

#7 - PRECISION: 0.4479
 

In [ ]:
models = [
    'best_lgbm_pr_auc_optimized_20251108_221353'
    'best_lgbm_pr_auc_optimized_20251106_215224',
    'best_lgbm_pr_auc_optimized_20251108_223541',
    'best_lgbm_pr_auc_optimized_20251108_225820',
    'best_lgbm_precision_optimized_20251108_232046',
    'best_lgbm_f1_optimized_20251108_235801',
    'best_lgbm_pr_auc_optimized_20251109_001812',
    'best_lgbm_pr_auc_optimized_20251109_004121',
    'best_lgbm_precision_optimized_20251109_012906',
    'best_lgbm_precision_optimized_20251109_021948', 
    'best_lgbm_precision_optimized_20251109_023236',
    'best_lgbm_precision_optimized_20251109_041233',
    'best_lgbm_precision_optimized_20251109_090643'
]

In [96]:
import joblib
loaded_model = joblib.load(MODEL_DIR / 'lgbm_borderline_smote_precision_20251111_022110.pkl')

In [97]:
# 테스트셋 성능 평가
y_pred = loaded_model.predict(X_test)
y_prob = loaded_model.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.433).astype(int)

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_pred, digits=3))

# Balanced Accuracy
bal_acc = balanced_accuracy_score(y_test, y_pred)
print(f"\nBalanced Accuracy (Test Set): {bal_acc:.3f}")

# ROC-AUC
roc_auc = roc_auc_score(y_test, y_prob)
pr_auc = average_precision_score(y_test, y_prob)
print(f"ROC-AUC (Test Set): {roc_auc:.3f}")
print(f"PR-AUC (Test Set): {pr_auc:.4f}")


Classification Report (Test Set):
              precision    recall  f1-score   support

           0      0.904     0.954     0.928      7550
           1      0.374     0.214     0.272       971

    accuracy                          0.870      8521
   macro avg      0.639     0.584     0.600      8521
weighted avg      0.844     0.870     0.854      8521


Balanced Accuracy (Test Set): 0.584
ROC-AUC (Test Set): 0.661
PR-AUC (Test Set): 0.2636


In [ ]:
thr_list = np.round(np.arange(0.1, 1.0, 0.01), 2)
rows = []
for thr in thr_list:
    y_hat = (y_prob >= thr).astype(int)
    rows.append([
        thr,
        precision_score(y_test, y_hat, zero_division=0),
        recall_score(y_test, y_hat, zero_division=0),
        f1_score(y_test, y_hat, zero_division=0)
    ])

thr_table = pd.DataFrame(rows, columns=['threshold', 'precision', 'recall', 'f1'])

fig = go.Figure()
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["precision"],
                         mode="lines", name="Precision",
                         line=dict(dash="dash", color="blue")))
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["recall"],
                         mode="lines", name="Recall",
                         line=dict(color="red")))
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["f1"],
                         mode="lines", name="F1",
                         line=dict(color="orange")))

fig.update_layout(title="임곗값 변화에 따른 Precision/Recall/F1",
                  xaxis_title="Threshold",
                  yaxis_title="Score",
                  yaxis=dict(range=[0,1]),
                #   template="plotly_dark"
)
fig.show()

In [ ]:
# ✅ pos_label=1 (가격미달) 기준으로 PR 곡선 계산
precisions, recalls, thresholds = precision_recall_curve(
    y_test, y_prob
)

# Recall 50% 보장
target_recall = 0.4
idx = np.where(recalls >= target_recall)[0]

if len(idx) > 0:
    best_idx = idx[np.argmax(precisions[idx])]
    optimal_threshold = thresholds[best_idx]
    
    print("\n" + "="*60)
    print(f"🎯 Recall {target_recall*100}% 보장 시 최적 임계값")
    print("="*60)
    print(f"최적 임계값: {optimal_threshold:.3f} (기본값 0.5)")
    print(f"Precision: {precisions[best_idx]:.3f}")
    print(f"Recall: {recalls[best_idx]:.3f}")

In [ ]:
import joblib
loaded_model = joblib.load(MODEL_DIR / 'best_lgbm_precision_optimized_20251109_041233.pkl')

params = loaded_model.named_steps['model'].get_params()

params['n_estimators'] = 1000000
params['learning_rate'] = 0.005
# params['num_leaves'] = 10
# params['max_depth'] = 5
# params['verbosity'] = 1
# params['metric'] = 'precision'

X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

display(params)

# pipeline = ImbPipeline([
#     ('model', LGBMClassifier(**params))
# ])

lgbm_model = LGBMClassifier(
    **params
)


def custom_precision(y_true, y_prob):
    y_pred = (y_prob >= 0.5).astype(int)
    return 'precision', precision_score(y_true, y_pred), True

lgbm_model.fit(
    X_train2, y_train2,
    eval_set=[(X_valid, y_valid)],  # train, valid 둘 다
    eval_names=['valid'],  # 이름 지정
    eval_metric=[custom_precision, 'average_precision'],  # PR-AUC (불균형 데이터에 적합)
    callbacks=[
        early_stopping(stopping_rounds=5000, verbose=True),
        log_evaluation(period=100)  # 100 iteration마다 출력
    ]
)


In [ ]:


# 테스트셋 성능 평가
y_pred = lgbm_model.predict(X_test)
y_prob = lgbm_model.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.512).astype(int)

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_pred, digits=3))

# Balanced Accuracy
bal_acc = balanced_accuracy_score(y_test, y_pred)
print(f"\nBalanced Accuracy (Test Set): {bal_acc:.3f}")

# ROC-AUC
roc_auc = roc_auc_score(y_test, y_prob)
pr_auc = average_precision_score(y_test, y_prob)
print(f"ROC-AUC (Test Set): {roc_auc:.3f}")
print(f"PR-AUC (Test Set): {pr_auc:.4f}")

In [ ]:
thr_list = np.round(np.arange(0.1, 1.0, 0.01), 2)
rows = []
for thr in thr_list:
    y_hat = (y_prob >= thr).astype(int)
    rows.append([
        thr,
        precision_score(y_test, y_hat, zero_division=0),
        recall_score(y_test, y_hat, zero_division=0),
        f1_score(y_test, y_hat, zero_division=0)
    ])

thr_table = pd.DataFrame(rows, columns=['threshold', 'precision', 'recall', 'f1'])

fig = go.Figure()
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["precision"],
                         mode="lines", name="Precision",
                         line=dict(dash="dash", color="cyan")))
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["recall"],
                         mode="lines", name="Recall",
                         line=dict(color="magenta")))
fig.add_trace(go.Scatter(x=thr_table["threshold"], y=thr_table["f1"],
                         mode="lines", name="F1",
                         line=dict(color="yellow")))

fig.update_layout(title="임곗값 변화에 따른 Precision/Recall/F1",
                  xaxis_title="Threshold",
                  yaxis_title="Score",
                  yaxis=dict(range=[0,1]),
                  template="plotly_dark")
fig.show()

In [ ]:
model = loaded_model.named_steps['model']

In [ ]:
import shap

# SHAP 적용
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# Plot
shap.summary_plot(shap_values, X_test)


In [ ]:
# mean absolute shap value per feature
shap_importance = np.abs(shap_values).mean(axis=0)
print(shap_importance)
shap_df = pd.DataFrame({
    'feature': X_test.columns,
    'shap_importance': shap_importance
}).sort_values(by='shap_importance', ascending=False)

print(shap_df)

## 집중학습

In [ ]:
# ============================================
# 2. Pipeline 구성
# ============================================

pipeline = ImbPipeline([
    ('model', LGBMClassifier(
        objective='binary',
        device='gpu',
        random_state=42,
        verbosity=-1,
        n_jobs=5,
        boost_from_average=False,
        # scale_pos_weight=scale_pos_weight,  # ✅ 자동으로 pos_label=1에 적용
        # bagging_freq=1,
        # pos_bagging_fraction=1,
        # neg_bagging_fraction=0.25,
        force_col_wise=True,
        metric='None',  # 🔥 추가: 기본 메트릭 끄기
    ))
])

print("\n✅ Pipeline 구성:")
print("   1. LGBMClassifier (GPU)")
print(f"      - scale_pos_weight={scale_pos_weight:.2f} (클래스 1=가격미달 가중치)")

# ============================================
# 3. 파라미터 분포
# ============================================
print(f"\n소수 클래스(1=가격미달) 샘플 수: {minority_count}개")

best_params = {
    "model__colsample_bytree": 0.6182968135250051,
    "model__learning_rate": 0.04421915627685693,
    "model__max_depth": 7,
    "model__min_child_samples": 16,
    "model__min_child_weight": 0.01,
    "model__n_estimators": 850,
    "model__num_leaves": 66,
    "model__reg_alpha": 0.3791315979645113,
    "model__reg_lambda": 0.012293458229400755,
    "model__scale_pos_weight": 5.199111963760975,
    "model__subsample": 0.7294440876598165
}

param_distributions = {
    # 🔥 LightGBM 파라미터 - 소수 클래스 민감도 향상
    'model__n_estimators': randint(800, 2000),
    'model__learning_rate': uniform(0.04, 0.05),
    'model__max_depth': randint(3, 6),
    'model__num_leaves': randint(15, 50),
    'model__min_child_samples': randint(20, 50),
    'model__subsample': uniform(0.7, 0.3),
    'model__colsample_bytree': uniform(0.6, 0.2),
    'model__min_child_weight': [0.01],
    'model__reg_alpha': uniform(0.3, 0.5),
    'model__reg_lambda': uniform(0, 0.1),
    
    # 불균형 처리
    'model__scale_pos_weight': uniform(5.0, 6.0),
    
    # 범주형 처리
    # 'model__cat_l2': uniform(5.0, 15.0), 
}


# ============================================
# 4. Scoring 설정
# ============================================

scoring = {
    'recall': make_scorer(recall_score),
    'f2': make_scorer(fbeta_score, beta=2),              
    'pr_auc': make_scorer(average_precision_score, response_method='predict_proba'),
    'precision': make_scorer(precision_score, zero_division=0),
    'f1': make_scorer(f1_score, zero_division=0, average='macro'),
    'roc_auc': 'roc_auc'                                           
}

main_score = 'precision'

print("\n✅ Scoring 전략:")
print(f"   메인 목표: {main_score.upper()}")
print(f"   타깃: 클래스 1 (가격미달) 검출 최적화")
print(f"   pos_label: 1 (기본값, 생략 가능)")

# ============================================
# 5. Stratified K-Fold
# ============================================
n_folds = 5
n_iter = 300
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)


print(f"\n교차검증 Folds: {n_folds}")
print(f"랜덤 샘플링 조합 수: {n_iter}개")
print(f"총 Fits: {n_iter} × {n_folds} = {n_iter * n_folds}")

# ============================================
# 6. RandomizedSearchCV 실행
# ============================================

print("\n" + "="*60)
print("🚀 RandomizedSearchCV 시작!")
print("="*60)

lgbm_random = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=cv,
    scoring=scoring,
    refit=main_score,
    verbose=3,
    random_state=42,
    n_jobs=5,
    error_score=np.nan 
)

start_actual = time.time()
lgbm_random.fit(X_train, y_train)
actual_time = time.time() - start_actual

# ============================================
# 7. 결과 출력
# ============================================
print("\n" + "="*60)
print("✅ 완료!")
print("="*60)
print(f"⏱️  소요 시간: {actual_time/60:.1f}분 ({actual_time:.0f}초)")

print(f"\n🏆 최고 {main_score.upper()} (CV): {lgbm_random.best_score_:.4f}")

print("\n📋 최적 파라미터:")
for key, value in sorted(lgbm_random.best_params_.items()):
    if isinstance(value, float):
        print(f"   {key}: {value:.4f}")
    else:
        print(f"   {key}: {value}")


In [ ]:

# ============================================
# 8. 전체 결과 비교
# ============================================
results_df = pd.DataFrame(lgbm_random.cv_results_)

scoring_cols = [f'mean_test_{score}' for score in scoring.keys()]

comparison_cols = [f'rank_test_{main_score}'] + scoring_cols + ['params']

print("\n" + "="*60)
print(f"📊 상위 10개 파라미터 조합 ({main_score.upper()} 기준)")
print("="*60)
top_10 = results_df[comparison_cols].sort_values(f'rank_test_{main_score}').head(10)

for idx, row in top_10.iterrows():
    rank = int(row[f'rank_test_{main_score}'])
    pr_auc = row[f'mean_test_pr_auc']
    recall = row['mean_test_recall']
    f2 = row['mean_test_f2']
    precision = row['mean_test_precision']
    f1 = row['mean_test_f1']
    
    print(f"\n#{rank} - {main_score.upper()}: {row[f'mean_test_{main_score}']:.4f} | Recall: {recall:.4f}")
    print(f"   F2: {f2:.3f} | Precision: {precision:.3f} | F1: {f1:.3f}")
    
    params = row['params']
    print(f"   Model: lr={params.get('model__learning_rate', 0):.4f}, "
          f"depth={params.get('model__max_depth', 0)}, "
          f"leaves={params.get('model__num_leaves', 0)}")

# ============================================
# 9. 테스트 세트 평가
# ============================================
print("\n" + "="*60)
print("🧪 테스트 세트 평가")
print("="*60)

best_model = lgbm_random.best_estimator_
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

# ✅ 클래스 1(가격미달)의 확률 추출
y_pred_proba_minority = y_pred_proba[:, 1]  # ✅ 인덱스 1 = 클래스 1 = 가격미달

print(classification_report(
    y_test, y_pred, 
    target_names=['0 (일반가, 다수)', '1 (가격미달, 소수)']
))

print("\n혼동 행렬:")
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm,
                     index=['실제_0 (일반가)', '실제_1 (가격미달)'],
                     columns=['예측_0 (일반가)', '예측_1 (가격미달)'])
print(cm_df)

# ✅ 혼동 행렬 해석 수정
# confusion_matrix 구조:
#              예측_0  예측_1
# 실제_0 (일반가)  TN      FP
# 실제_1 (가격미달)   FN      TP

tn, fp, fn, tp = cm.ravel()

print("\n💼 비즈니스 해석 (소수 클래스=1=가격미달 기준):")
print(f"   ✅ 정확한 분류: {tn + tp}개 ({(tn+tp)/len(y_test)*100:.1f}%)")
print(f"   ⚠️  오탐 (FP): {fp}개 → 일반가을 가격미달로 오판 (재검사 필요)")
print(f"   🚨 미탐 (FN): {fn}개 → 가격미달를 일반가으로 오판 (치명적!)")

# ✅ Recall 계산: TP / (TP + FN)
if tp + fn > 0:
    recall_pct = tp/(tp+fn)*100
    print(f"\n   📊 소수 클래스(1=가격미달) Recall: {recall_pct:.1f}%")
    print(f"      → 가격미달 {tp+fn}개 중 {tp}개 검출 ({fn}개 놓침)")
else:
    print(f"\n   📊 소수 클래스(1=가격미달) Recall: N/A")

# ✅ Precision 계산: TP / (TP + FP)
if tp + fp > 0:
    precision_pct = tp/(tp+fp)*100
    print(f"   📊 소수 클래스(1=가격미달) Precision: {precision_pct:.1f}%")
    print(f"      → 가격미달 판정 {tp+fp}개 중 {tp}개 정답")
else:
    print(f"   📊 소수 클래스(1=가격미달) Precision: N/A")

test_roc_auc = roc_auc_score(y_test, y_pred_proba_minority)
print(f"   📊 ROC-AUC: {test_roc_auc:.4f}")

# ============================================
# 10. 🔥 임계값 조정으로 추가 개선
# ============================================
from sklearn.metrics import precision_recall_curve, average_precision_score

# ✅ pos_label=1 (가격미달) 기준으로 PR 곡선 계산
precisions, recalls, thresholds = precision_recall_curve(
    y_test, y_pred_proba_minority
)

# Recall 50% 보장
target_recall = 0.5
idx = np.where(recalls >= target_recall)[0]

if len(idx) > 0:
    best_idx = idx[np.argmax(precisions[idx])]
    optimal_threshold = thresholds[best_idx]
    
    print("\n" + "="*60)
    print(f"🎯 Recall {target_recall*100}% 보장 시 최적 임계값")
    print("="*60)
    print(f"최적 임계값: {optimal_threshold:.3f} (기본값 0.5)")
    print(f"Precision: {precisions[best_idx]:.3f}")
    print(f"Recall: {recalls[best_idx]:.3f}")
    
    # ✅ 임계값 조정: 확률 >= threshold이면 클래스 1(가격미달)로 예측
    y_pred_adjusted = (y_pred_proba_minority >= optimal_threshold).astype(int)
    
    cm_adj = confusion_matrix(y_test, y_pred_adjusted)
    tn2, fp2, fn2, tp2 = cm_adj.ravel()
    
    print(f"\n조정 효과:")
    print(f"   미탐(FN) 감소: {fn} → {fn2} (개선: {fn-fn2}개)")
    print(f"   오탐(FP) 증가: {fp} → {fp2} (증가: {fp2-fp}개)")
    
    if tp + fn > 0 and tp2 + fn2 > 0:
        print(f"   Recall 개선: {tp/(tp+fn)*100:.1f}% → {tp2/(tp2+fn2)*100:.1f}%")
    
    if fn > fn2:
        print(f"   💡 가격미달를 일반가으로 오판하는 치명적 오류 {fn-fn2}개 감소!")
else:
    print(f"\n⚠️  Recall {target_recall*100}% 달성 불가 (최대 Recall: {recalls.max():.2%})")

# ============================================
# 11. 변수 중요도 확인
# ============================================
print("\n" + "="*60)
print("📊 변수 중요도 Top 10")
print("="*60)

final_model = best_model.named_steps['model']
feature_names = list(X_train.columns)

n_features_used = len(final_model.feature_importances_)
feature_names = feature_names[:n_features_used]

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print(importance_df.head(10).to_string(index=False))

# ============================================
# 12. 교차검증 vs 테스트 성능 비교
# ============================================
print("\n" + "="*60)
print("📈 성능 비교: 교차검증 vs 테스트")
print("="*60)

# ✅ pos_label=1 기본값 사용
test_metrics = {
    'PR-AUC': average_precision_score(y_test, y_pred_proba_minority),
    'Recall': recall_score(y_test, y_pred),
    'F2': fbeta_score(y_test, y_pred, beta=2),
    'Precision': precision_score(y_test, y_pred, zero_division=0),
    'F1': f1_score(y_test, y_pred, zero_division=0),
    'ROC-AUC': test_roc_auc
}

print(f"{'지표':<15} {'교차검증 (CV)':<20} {'테스트':<15} {'차이':<10}")
print("-" * 60)

for metric in ['PR-AUC', 'Recall', 'F2', 'Precision', 'F1', 'ROC-AUC']:
    cv_key = metric.lower().replace('-', '_')
    cv_score = results_df.loc[results_df[f'rank_test_{main_score}'] == 1, f'mean_test_{cv_key}'].values[0]
    test_score = test_metrics[metric]
    diff = test_score - cv_score
    
    diff_str = f"{diff:+.4f}"
    if abs(diff) > 0.05:
        diff_str += " ⚠️"
    
    print(f"{metric:<15} {cv_score:<20.4f} {test_score:<15.4f} {diff_str}")

# ============================================
# 13. 최종 요약
# ============================================
print("\n" + "="*60)
print(f"📝 최종 요약 ({main_score.upper()} 최적화)")
print("="*60)
print(f"✅ 최적 모델 {main_score.upper()} (CV): {lgbm_random.best_score_:.4f}")
print(f"✅ 테스트 Recall: {test_metrics['Recall']*100:.1f}% (가격미달 검출률)")
print(f"✅ 테스트 Precision: {test_metrics['Precision']*100:.1f}% (가격미달 판정 정확도)")
print(f"✅ 테스트 F1: {test_metrics['F1']:.4f}")
print(f"✅ 치명적 오류(미탐 FN): {fn}개 / {tp+fn}개")
print(f"⏱️  총 소요 시간: {actual_time/60:.1f}분")

print("\n💡 핵심 포인트:")
print(f"   - 가격미달(1) {tp+fn}개 중 {tp}개 검출 ({tp/(tp+fn)*100:.1f}%)")
print(f"   - 놓친 가격미달: {fn}개")
print(f"   - 오탐 (일반가→가격미달): {fp}개 (재검사 필요)")

# ============================================
# 14. 모델 + 파라미터 저장
# ============================================
print("\n" + "="*60)
print("💾 모델 및 파라미터 저장")
print("="*60)

save_model_input = input("모델을 저장하시겠습니까? (y/n): ")
if save_model_input.lower() == 'y':
    
    # MODEL_DIR이 정의되지 않은 경우 현재 디렉토리 사용
    try:
        MODEL_DIR = Path(MODEL_DIR)
    except:
        MODEL_DIR = Path('.')
        print(f"⚠️  MODEL_DIR 미정의, 현재 디렉토리 사용: {MODEL_DIR.absolute()}")
    
    MODEL_DIR.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    model_filename = MODEL_DIR / f'best_lgbm_{main_score}_optimized_{timestamp}.pkl'
    params_filename = MODEL_DIR / f'best_params_{main_score}_{timestamp}.json'
    metrics_filename = MODEL_DIR / f'test_metrics_{main_score}_{timestamp}.json'
    results_filename = MODEL_DIR / f'cv_results_{main_score}_{timestamp}.csv'
    
    # 1. 모델 저장
    joblib.dump(best_model, model_filename)
    print(f"✅ 모델 저장: {model_filename}")
    
    # 2. 파라미터 저장
    best_params_serializable = {}
    for key, value in lgbm_random.best_params_.items():
        if isinstance(value, (np.integer, np.floating)):
            best_params_serializable[key] = float(value) if isinstance(value, np.floating) else int(value)
        else:
            best_params_serializable[key] = value
    
    params_info = {
        'best_params': best_params_serializable,
        f'best_{main_score}_score': float(lgbm_random.best_score_),
        'optimization_target': main_score,
        'cv_folds': n_folds,
        'n_iter': n_iter,
        'timestamp': timestamp,
        'class_labels': {
            '0': '일반가 (다수)',
            '1': '가격미달 (소수)'
        },
        'minority_count': int(minority_count),
        'majority_count': int(majority_count),
        'scale_pos_weight': float(scale_pos_weight),
        'features': features,
        'train_test_split': split_date
    }
    
    with open(params_filename, 'w', encoding='utf-8') as f:
        json.dump(params_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 파라미터 저장: {params_filename}")
    
    # 3. 테스트 성능 저장
    test_metrics_serializable = {
        key: float(value) if isinstance(value, (np.floating, np.integer)) else value
        for key, value in test_metrics.items()
    }
    
    metrics_info = {
        'test_metrics': test_metrics_serializable,
        'confusion_matrix': {
            'tn': int(tn),  # 일반가 → 일반가 (정답)
            'fp': int(fp),  # 일반가 → 가격미달 (오탐)
            'fn': int(fn),  # 가격미달 → 일반가 (미탐, 치명적!)
            'tp': int(tp)   # 가격미달 → 가격미달 (정답)
        },
        'business_interpretation': {
            'total_minority': int(tp + fn),      # 실제 가격미달 개수
            'detected_minority': int(tp),        # 검출된 가격미달
            'missed_minority': int(fn),          # 놓친 가격미달 (치명적)
            'recall_percentage': float(tp/(tp+fn)*100) if (tp+fn) > 0 else 0,
            'false_alarms': int(fp)              # 일반가을 가격미달로 오판
        },
        'timestamp': timestamp
    }
    
    with open(metrics_filename, 'w', encoding='utf-8') as f:
        json.dump(metrics_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 테스트 성능 저장: {metrics_filename}")
    
    # 4. CV 결과 저장
    results_df.to_csv(results_filename, index=False, encoding='utf-8')
    print(f"✅ CV 결과 저장: {results_filename}")
    
    print("\n📦 저장된 파일 요약:")
    print(f"1. 모델: {model_filename.name}")
    print(f"2. 파라미터: {params_filename.name}")
    print(f"3. 성능: {metrics_filename.name}")
    print(f"4. CV결과: {results_filename.name}")
    
else:
    print("모델 저장 생략")

print("\n" + "="*60)
print("🎉 모든 작업 완료!")
print("="*60)